In [8]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [9]:
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

#zips two arrays into a tuple object
lat_lngs = zip(lats, lngs)
lat_lngs

In [10]:
coordinates = list(lat_lngs)

In [13]:
cities = []

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    #checks if the city isn't already on the list
    if city not in cities:
        cities.append(city)

len(cities)
        

631